<a href="https://colab.research.google.com/github/nicholasdcrotty/CMBG_BRM_CNNOculomotorAnalysis/blob/main/analysisForReplication/CMBG_BHR_CNNBasedAnalysis_ForReplication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# Using CNNs to analyze oculomotor timecourse data - Replicating observed results from Crotty, Massa, Benson, & Grubb


Before running this code, make sure to select a fast runtime (either your own local runtime or one provided by Colab), as the model fitting procedure and SHAP analysis takes quite some time on the CPU alone. **NOTE: Due to the inherent stochasticty in the neural network fitting process, the resulting classification accuracy may be slightly different from the values reported in the manuscript.**




**Before running this code**, you need to load the data files from the corresponding studt into the Colab environment. To do so, click the file icon on the left sidebar, click the "Upload to session storage" icon (the page icon with the upwards arrow) and upload the matching features and conditions files. There should be two feature `.csv` files and one conditions `.csv`. file

# Load in necessary libraries

In [ ]:
# libraries related to neural network
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

# libraries related to input/output arrays
import numpy as np
import pandas as pd

#libraries related to importing data into script
from google.colab import files

#libraries related to graphing NN results
import matplotlib.pyplot as plt
import seaborn as sns

#SHAP values
!pip install shap
import shap

# Indicate which set of results to replicate here!

In [ ]:
potentialReplications = ["massaTarget", "grubbLiTarget", "massaDistractor", "grubbLiDistractor"] #choose from one of these four
replicateAnalysis = "grubbLiDistractor"

#Update script with correct experimental details

In [ ]:
#details for first section of results - CNN predicting target location from Massa et al. (2024) dataset
if replicateAnalysis == "massaTarget":
  details = {"desiredLabel": "targetLocation", "inputFeatures" : 2, "outputClasses" : 6, "removeNAs" : False, "removeRowLabels" : True, "numericallyEncode" : True}
  print("CNN predicting target location from Massa et al. (2024) dataset")

#details for second section of results - CNN predicting target location from Grubb & Li (2018) dataset
elif replicateAnalysis == "grubbLiTarget":
  details = {"desiredLabel": "targetLocation", "inputFeatures" : 2, "outputClasses" : 6, "removeNAs" : False, "removeRowLabels" : True, "numericallyEncode" : True}
  print("CNN predicting target location from Grubb & Li (2018) dataset")

#details for third section of results - CNN predicting distractor location from Massa et al. (2024) dataset
elif replicateAnalysis == "massaDistractor":
  details = {"desiredLabel": "valueDistractorLocation", "inputFeatures" : 2, "outputClasses" : 6, "removeNAs" : True, "removeRowLabels" : True, "numericallyEncode" : True}
  print("CNN predicting distractor location from Massa et al. (2024) dataset")

#details for fourth section of results - CNN predicting distractor location from Grubb & Li (2018) dataset
elif replicateAnalysis == "grubbLiDistractor":
  details = {"desiredLabel": "distractorLocation", "inputFeatures" : 2, "outputClasses" : 6, "removeNAs" : True, "removeRowLabels" : True, "numericallyEncode" : True}
  print("CNN predicting distractor location from Grubb & Li (2018) dataset")

# Load in data

In [ ]:
#upload = files.upload() #The GUI is actually much faster at this

#load in data if replicating analyses using Massa et al. (2024) dataset
if replicateAnalysis == "massaTarget" or replicateAnalysis == "massaDistractor":
  feature1 = pd.read_csv('CMBG_BRM_Section4_XPos.csv')
  feature2 = pd.read_csv('CMBG_BRM_Section4_YPos.csv')
  conditions = pd.read_csv('CMBG_BRM_Section4_Conditions.csv')
  print("Massa dataset read")

#load in data if replicating analyses using Grubb & Li (2018) dataset
elif replicateAnalysis == "grubbLiTarget" or replicateAnalysis == "grubbLiDistractor":
  feature1 = pd.read_csv('CMBG_BRM_Section2_XPos.csv')
  feature2 = pd.read_csv('CMBG_BRM_Section2_YPos.csv')
  conditions = pd.read_csv('CMBG_BRM_Section2_Conditions.csv')
  print("Grubb & Li dataset read")


#format data into 3d numpy array
if (details["inputFeatures"] == 1):
  arr2d = feature1.values
  data = arr2d[:, np.newaxis,:]
else:
  data = np.stack((feature1, feature2), axis=1)
#remove row labels if needed
if details["removeRowLabels"] == True:
  data = data[:,:,1:]

print(data.shape)

#Extract desired labels from conditions datafile

In [ ]:
label = details['desiredLabel']

#remove NA values if necessary (e.g., if search object is absent on some trials)
if details["removeNAs"] == True:
  data = data[conditions[label].isna()==False, :, :]
  conditions = conditions[conditions[label].isna()==False]

#extract label information from conditions file - this will serve as the comparison during supervised learning
label = details['desiredLabel']
labels = conditions[label].astype(int)

#numerical encoding of labels -- The actual method of encoding can change based on your data.
#the example encoding is what we used for our reported analyses.
if details["numericallyEncode"] == True:
  labels = labels.map({0:0, 60:1, 120:2, 180:3, 240:4, 300:5})

In [ ]:
print(labels.head())

#Training-validation split (66% training, 33% validation)

In [ ]:
N, R, C = data.shape

#find number of training/test trials (if data is not divisible by 3, add extras to training set)
n_test = N // 3 #same as floor(N/3)
n_train = N - n_test

#predefine dataframes with corresponding trial lengths
training_data = np.zeros((n_train, R, C))
test_data = np.zeros((n_test, R, C))

training_labels = []
test_labels = []

trainCounter = 0
testCounter = 0

for i in range(0, len(data)):
  if i % 3 == 0: #every third trial, add data and label to validation set
    test_data[testCounter, :, : ] = data[i, :, :]
    test_labels.append(labels.iloc[i])
    testCounter += 1
  else: #otherwise, add data and label to training set
    training_data[trainCounter, :, : ] = data[i, :, :]
    training_labels.append(labels.iloc[i])
    trainCounter += 1

training_labels = pd.Series(training_labels)
test_labels = pd.Series(test_labels)

print(training_data.shape, test_data.shape)

# Pre-process data

In [ ]:
#transform numpy data to tensor
training_data = torch.from_numpy(training_data)
test_data = torch.from_numpy(test_data)

#change to float type - more easily handled by training + test loops
training_data = training_data.float()
test_data = test_data.float()

#Check dimensions of test data and labels

In [ ]:
print(test_data.shape)
#trials, features, samples

print(test_labels.shape)
# 1-dimensional tensor of identical length to rows of test data

#Determine size of flattened output passed to first linear transform -- different based on the length of your trial and number of features

In [ ]:
sizeChecker = nn.Sequential( #the same layers as the first portion of the CNN, just to see what that layer's output size will be
        nn.Conv1d(in_channels = details["inputFeatures"], out_channels = 64, kernel_size = 3),
        nn.ReLU(),
        nn.Dropout(0.25),
        nn.MaxPool1d(kernel_size=5),
        nn.Flatten()
        )
tmp = sizeChecker(test_data[0:1,:,:])
flattenOutput = tmp.shape[1]
print(flattenOutput)

# Initialize the neural network


In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        #Flattening layer (gets called after Sequential convolution layer)
        self.flatten = nn.Flatten()

        #Sequential layer containing convolution and subsequent regularization methods
        self.cnnL1 = nn.Sequential(
        nn.Conv1d(in_channels = details["inputFeatures"], out_channels = 64, kernel_size = 3),
        nn.ReLU(),
        nn.Dropout(0.25),
        nn.MaxPool1d(kernel_size=5),
        )

        #Linear transforms
        self.lin = nn.Linear(in_features = flattenOutput, out_features = 64)
        self.lin2 = nn.Linear(in_features = 64, out_features = 32)
        self.lin3 = nn.Linear(in_features = 32, out_features = details["outputClasses"]) #Output is X number of logits, to be used as the CNN's predictions with loss function

    def forward(self, x):
        #apply convolution
        conv = self.cnnL1(x)

        #rearrage resulting array for proper order during flattening
        permuted = conv.permute(0,2,1)

        #flatten array into one-dimensional tensor
        flattened = self.flatten(permuted)

        #linear transforms, with logits as output
        linear = self.lin(flattened)
        linear2 = self.lin2(linear)
        logits = self.lin3(linear2)
        return logits

model = NeuralNetwork()

# Initialize Early Stopping class

In [ ]:
#EarlyStopping class used in pytorchtools library
#source: https://github.com/Bjarten/early-stopping-pytorch/blob/master/pytorchtools.py
class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pth', trace_func=print):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement.
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
            path (str): Path for the checkpoint to be saved to.
                            Default: 'checkpoint.pt'
            trace_func (function): trace print function.
                            Default: print
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.inf
        self.delta = delta
        self.path = path
        self.trace_func = trace_func
    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            self.trace_func(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

# Define a Custom Dataset object that stores the samples and the labels together for each trial

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, labelsObject, dataObject, transform=None, target_transform=None):
        self.labels = labelsObject
        self.dataframe = dataObject
        self.transform = transform #in case data needs to be transformed to a different type
        self.target_transform = target_transform #in case labels need to be transformed to a different type

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        data = self.dataframe[idx]
        label = self.labels.iloc[idx]
        if self.transform:
            data = self.transform(data)
        if self.target_transform:
            label = self.target_transform(label)
        return data, label

# Create the CustomDatasets for training and validation data

In [ ]:
train = CustomDataset(labelsObject = training_labels, dataObject = training_data)
test = CustomDataset(labelsObject = test_labels, dataObject = test_data)

# Create DataLoader objects containing the training and test data/labels

In [ ]:
batch_size = 64
train_dataloader = DataLoader(train, batch_size=batch_size, drop_last=False)
test_dataloader = DataLoader(test, batch_size=batch_size, drop_last=False)

## Hyperparameters & Optimizer




In [ ]:
#Hyperparameters
learning_rate = 1e-3
epochs = 2500 #arbitrary large number to give time for early stopping to occur
loss_fn = nn.CrossEntropyLoss()

#Adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


## Full Implementation



In [ ]:
def train_model(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train() #set the model to training mode - important for batch normalization and dropout layers
    for batch, (X, y) in enumerate(dataloader):

        #compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        #backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        #print current loss every 20 batches
        if batch % 20 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_model(dataloader, model, loss_fn, trialLevelDF, epNum = 0, earlyStop="none", lossChecker = "none", mode = "loss"):
    size = len(dataloader.dataset)
    model.eval() #set the model to evaluation mode - important for batch normalization and dropout layers

    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    #evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    with torch.no_grad():
      batchCount = 0
      for X, y in dataloader:
          pred = model(X) #apply model to batch
          predSize = len(pred)
          for p in (range(predSize)): #iterate through current batch to get trial-level accuracies

              #determine whether there was a correct or incorrect prediction on the current dataframe
              trialAcc = (pred[p].argmax().item() == y[p].item())

              #save trial-level accuracy to initialized dataframe
              trialLevelDF.iloc[(batch_size*batchCount)+p,epNum] = trialAcc

          #update loss and accuracy metrics for reporting
          test_loss += loss_fn(pred, y).item()
          correct += (pred.argmax(1) == y).type(torch.float).sum().item() #same calculation as above, but performed for entire batch at once
          batchCount+=1

    #compute overall loss and accuracy
    test_loss /= num_batches
    correct /= size

    #if we want to save the CNN's lowest loss value, record that minimum loss and the epoch where it occurred
    if lossChecker != "none" and test_loss < lossChecker[0]:
      lossChecker[0] = test_loss
      lossChecker[1] = epNum

    #if we want to implement early stopping, apply early stopping
    if earlyStop != "none":
      earlyStop(test_loss, model)

    #print the relevant performance metrics -- loss, accuracy, or both
    if mode == "loss":
      print(f"Epoch {epNum+1} complete! \n Current loss: {test_loss:>8f}    Lowest loss: {lossChecker[0]:>8f} \n")
    elif mode == "accuracy":
      print(f"Accuracy: {(100*correct):>8f}% \n")
    elif mode =="both":
      print(f"Epoch {epNum+1} complete! \n Current loss: {test_loss:>8f}    Lowest loss: {lossChecker[0]:>8f} \n Accuracy: {(100*correct):>8f}% \n")

# Implement training and test procedures


In [ ]:
#for storing CNN validation accuracy in each epoch
performanceSummary = pd.DataFrame(index=range(len(test_dataloader.dataset)), columns = range(epochs))

#early stopping object
early_stopping = EarlyStopping(patience=10, verbose=True)

#for storing minimum loss value and epoch where it occurred
lossCheck = [np.inf, 0]

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")

    #training
    train_model(train_dataloader, model, loss_fn, optimizer)

    #validation
    test_model(test_dataloader, model, loss_fn, performanceSummary,
               epNum = t, earlyStop = early_stopping, lossChecker=lossCheck, mode = "loss")

    #check early stopping criteria
    if early_stopping.early_stop:
      print(f"Early stopping @ epoch {t+1}")
      break

print("Done!")

# Quick summary of model results

In [ ]:
import matplotlib.pyplot as plt
bestEpoch = lossCheck[1]
performanceSummary = performanceSummary.iloc[:,0:t+1] #in case early stopping happens
epAcc = (100*(performanceSummary.sum() / len(test_dataloader.dataset)))
bestPerformance = epAcc[bestEpoch]

#plot accuracy across epochs and label accuracy from interation with lowest validation loss
plt.plot(range(t+1), epAcc)
plt.arrow(x = bestEpoch, y = bestPerformance - 0.1*bestPerformance, dx = 0, dy = 0.1*bestPerformance, width = .2)
plt.text(x = bestEpoch-0.33*bestEpoch, y = bestPerformance - 0.2*bestPerformance, s = f"Accuracy from epoch {bestEpoch} (starting \n at 0) with the lowest loss -- \n we use this in analysis!")
plt.text(x = bestEpoch-0.33*bestEpoch, y = bestPerformance - 0.3*bestPerformance, s = "Note: Overfitting may have \n happened in later epochs, \n causing a severe accuracy \n decrease")

#save graph of performance across epochs
plt.savefig(".png")
files.download(".png")

#view graph and accuracy values across epochs
plt.show()
print(epAcc)

# View accuracy from epoch with lowest loss

In [ ]:
#load in best weights
bestWeights = torch.load("checkpoint.pth")
model.load_state_dict(bestWeights)

#check of prediction accuracy for model
trialLevelAcc = pd.DataFrame(index=range(len(test_dataloader.dataset)), columns = range(1))
test_model(test_dataloader, model, loss_fn, trialLevelAcc, mode = "accuracy")

#Download trial level accuracy as .csv file through Google Drive

In [ ]:
trialLevelAcc.to_csv('.csv')
files.download('.csv')

# Save model parameter weights locally and download to computer

In [ ]:
torch.save(model.state_dict(), ".pth") #.pth file is type used for storing parameter weights
files.download(".pth")

#View exact accuracy of the model from the corresponding results section
If you would like to see the performance of the model that produced the classifcation accuracy reported in the corresponding results section, run the code below. **Prior to running this code,** upload the corresponding `.pth` file to the Colab environment.

In [ ]:
#to upload weights from previous model
if replicateAnalysis == "massaTarget"
  weightsName = "model_state_MassaTargLoc.pth"

elif replicateAnalsis == "grubbLiTarget"
  weightsName = "model_state_GrubbLiTargLoc.pth

elif replicateAnalysis == "massaDistractor"
  weightsName = "model_state_MassaDistLoc.pth"

elif replicateAnalysis == "grubbLiDistractor"
  weightsName = "model_state_GrubbLiDistLoc.pth"

#load in weights
modelWeights = torch.load(weightsName)
model.load_state_dict(modelWeights)

#check accuracy
tmp = pd.DataFrame(index=range(len(test_dataloader.dataset)), columns = range(1))
test_model(test_dataloader, model, loss_fn, tmp, mode = "accuracy")

#Calculate SHAP values for each trial of validation set
Fair warning, this takes a substantial amount of time to run.

In [ ]:
trialLevelSHAP = pd.DataFrame(index=range(len(test_dataloader.dataset)), columns = range(len(test_dataloader.dataset[0][0][0])))
print(trialLevelSHAP.shape)

shapBatch = test_dataloader.dataset[0:100] #first 100 trials used for explanation model
explainer = shap.DeepExplainer(model, shapBatch[0])

loopDur = len(test_dataloader.dataset)
for s in range(loopDur):

  #calculate SHAP values relative to explainer set
  shap_values = explainer.shap_values(test_dataloader.dataset[s:s+1][0])

  #compute global feature importance by taking absolute value
  abs_shap_values = np.abs(shap_values)

  #average across classes and features to get one global feature importance value per sample
  SHAP = abs_shap_values.mean(axis=(0,1,3))

  #save trial-level accuracy to initialized dataframe
  trialLevelSHAP.iloc[s,:] = SHAP
  if s % 100 ==0:
    print(s)

print("Done!")

#Download SHAP values as .csv file

In [ ]:
trialLevelSHAP = pd.DataFrame(trialLevelSHAP)
trialLevelSHAP.to_csv('.csv')
files.download('.csv')

In [ ]:
#Graphing heatmap of results for last trial as a quick check
df = pd.DataFrame({"SHAP values for each sample in timecourse": abs_shap_values.mean(axis=(0,1,3))},
                  index=range(len(shap_values[0][0])))
g = sns.heatmap(df, fmt="g", cmap='viridis')
g.set_yticks(range(0,600,50))
g.set_yticklabels(range(0,600,50))
plt.show()